[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1QTztBzC49lJgPu0KaqyakoRRuAfqLPyd#scrollTo=1Shko6DE2zXY)
      

In [1]:
# !pip install yfinance

In [2]:
from typing import Optional

import numpy as np
import pandas as pd
import plotly
import scipy.stats as sts
import yfinance as yf
from plotly.subplots import make_subplots

plotly.offline.init_notebook_mode(connected=False)

## Загрузка данных
Возьмем тикеры Hang Seng - биржевой индекс Гонконгской фондовой биржи.

1. HSBC - один из крупнейших финансовых конгломератов в мире [https://ru.wikipedia.org/wiki/HSBC](https://ru.wikipedia.org/wiki/HSBC)
2. 0941.HK (China Mobile Limited) - китайская телекоммуникационная компания [https://ru.wikipedia.org/wiki/China_Mobile](https://ru.wikipedia.org/wiki/China_Mobile)
3. 0939.HK (China Construction Bank) - один из крупнейших банков Китая и мира [https://ru.wikipedia.org/wiki/China_Construction_Bank](https://ru.wikipedia.org/wiki/China_Construction_Bank)
4. 1398.HK (Industrial and Commercial Bank of China) - крупнейший коммерческий банк Китая и мира. [https://ru.wikipedia.org/wiki/Industrial_and_Commercial_Bank_of_China](https://ru.wikipedia.org/wiki/Industrial_and_Commercial_Bank_of_China)
5. 0883.HK (China National Offshore Oil Corporation) - третья по величине национальная нефтяная компания Китая. [https://ru.wikipedia.org/wiki/China_National_Offshore_Oil_Corporation](https://ru.wikipedia.org/wiki/China_National_Offshore_Oil_Corporation)
6. TCEHY (Tencent) - китайская инвестиционная холдинговая компания. [https://ru.wikipedia.org/wiki/Tencent](https://ru.wikipedia.org/wiki/Tencent)


In [3]:
tickers = ['HSBC', '0941.HK', '0939.HK', '1398.HK', '0883.HK', 'TCEHY']  # Список тикеров
dataframes = []  # Список в который сохраним датасеты

for i in range(len(tickers)):
    # Мы используем функцию yf.download. Эта функция скачивает с yahoo finance котировки тикеров
    # возвращает массив со столбцами 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'.
    # И в индексе расположена дата
    ticker_data = yf.download(tickers[i], start='2012-01-01', end='2022-01-01', interval='1d').reset_index()
    # Удалим те записи, в которых цена закрытия пустая
    ticker_data = ticker_data.drop(index=ticker_data[ticker_data['Adj Close'].isna()].index)
    # Добавим в список данных по тикерам, очередной тикер
    dataframes.append(ticker_data)
    
dataframes[0].head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2012-01-03,38.840000,39.520000,38.830002,39.230000,22.909386,2485500
1,2012-01-04,39.139999,39.279999,38.740002,39.250000,22.921070,1720700
2,2012-01-05,38.759998,38.860001,38.400002,38.799999,22.658278,2005100
3,2012-01-06,38.610001,38.610001,38.160000,38.310001,22.372131,1361700
4,2012-01-09,38.270000,38.290001,37.860001,38.259998,22.342930,1996600


In [4]:
plots_exchange = make_subplots(2, 3, subplot_titles=tickers, x_title='Дата', y_title='Цена')
for i in range(6):
    df_i = dataframes[i]
    plots_exchange.add_scatter(x=df_i['Date'], y=df_i['Adj Close'], showlegend=False,
                               row=i // 3 + 1, col=i % 3 + 1, marker={'color': 'rgb(40, 40, 200)'})

plots_exchange.update_layout(title={'text': 'Цены тикеров на даты', 'font': {'size': 28}})
plots_exchange.show(renderer="colab")

## Количество торговых дней для каждого тикера

In [5]:
def count_workdays(df: pd.DataFrame) -> int:
    """ Находит количество рабочих дней для тикера """
    _days: pd.Series = df.Date  # Забираем даты
    return len(np.unique(_days.dt.date))  # Находим длину массива уникальных дат


def agg_workdays_per_year(df: pd.DataFrame) -> pd.Series:
    """ Группирует по годам и считает количество рабочих дней """
    agg_per_year: pd.Series(index=pd.DatetimeIndex)  # Используем типизацию, чтобы среда понимала типы данных

    # Группируем по столбцу Date, агрегируя по годам и применяем функцию count_workdays, чтобы посчитать дни
    agg_per_year = df.groupby(pd.Grouper(key='Date', freq='1y')).apply(count_workdays)
    agg_per_year.index = agg_per_year.index.year  # Берем год и заменяем дату на просто год
    agg_per_year = agg_per_year.rename_axis('Год')  # Ставим название столбца индексов
    agg_per_year.name = tickers[i] + ' ' + 'Количество рабочих дней'  # Переименовываем столбец с количеством дней
    return agg_per_year


cnt_work_days = []  # Сохраним в этот список датафреймы после функции agg_workdays_per_year
for i in range(len(dataframes)):
    cnt_work_days.append(agg_workdays_per_year(dataframes[i]))

amount_workdays_per_year = pd.DataFrame(cnt_work_days).T
amount_workdays_per_year.head()

,HSBC Количество рабочих дней,0941.HK Количество рабочих дней,0939.HK Количество рабочих дней,1398.HK Количество рабочих дней,0883.HK Количество рабочих дней,TCEHY Количество рабочих дней
Год,,,,,,
2012,250,246,246,246,246,250
2013,252,245,245,245,245,252
2014,252,247,247,247,247,252
2015,252,247,247,247,247,252
2016,252,247,247,247,247,252


## Логарифмическая доходность
Логарифмическая доходность $r_i$ c момента $t_0$ до момента $t_1$, определяется как:
$$ r_i = \ln\left(\frac{P_{t_1}}{P_{t_0}}\right) = \ln(P_{t_1}) - \ln(P_{t_0}) $$

где $P_{t_i}$ - цена актива на момент $i$

Будем проверять нормальность логарифмической доходности для цен закрытия двух рабочих дней $i$ и $i + 1$.

In [6]:
def log_diff(prices: pd.Series) -> pd.Series:
    """ Вычисляет логарифмическое приращение приращение в массиве по формуле: ln(p_i) - ln(p_i+1)"""
    # Создаем пустой массив такого же размера, в который запишем доходности
    diff_prices = pd.Series([np.nan] * len(prices), name='Логарифмическая доходность', index=prices.index)
    # Вычисляем логарифмическую доходность
    log_profit_for_tickers = np.log(prices.values[1:]) - np.log(prices.values[:-1])
    # Записываем доходности в созданный массив
    diff_prices[1:] = log_profit_for_tickers
    return diff_prices


def log_transform_dataset(df: pd.DataFrame) -> pd.DataFrame:
    """ Трансформируем датасет, оставляя в нем логарифмическую доходность, объем на предыдущую дату и дату """
    df['Логарифмическая доходность'] = log_diff(df['Adj Close'])  # Находим логарифмическую доходность
    df['Объем на предыдущую дату'] = pd.Series([np.nan] + df['Volume'].iloc[:-1].to_list())  # Формируем объем
    df['Дата'] = df['Date']  # Переименовываем дату на русский язык
    return df[['Дата', 'Объем на предыдущую дату', 'Логарифмическая доходность']]


transformed_datasets = []
for i in range(len(dataframes)):
    transformed_datasets.append(log_transform_dataset(dataframes[i]))
    
transformed_datasets[0].head()

,Дата,Объем на предыдущую дату,Логарифмическая доходность
0,2012-01-03,NaN,NaN
1,2012-01-04,2485500.0,0.000510
2,2012-01-05,1720700.0,-0.011531
3,2012-01-06,2005100.0,-0.012709
4,2012-01-09,1361700.0,-0.001306


## Выделение определенного объема торгов накануне
Будем проверять, чтобы объем торгов накануне попадал в следующий интервал:
$$[Q_1 - k \cdot (Q_3 - Q_1), Q_3 + k \cdot (Q_3 - Q_1)]$$
где $Q_1, Q_3$ - первый и третий квартиль распределения объема торгов. $k$ - варьируемый параметр, примем $k = 0$.

Если объем торгов попал в этот интервал, то мы будем считать объем средним, если выше большим, ниже - низким.

In [7]:
def tukey_fences(volumes_: pd.Series, k_: float = 0) -> tuple:
    """ Возвращает левую и правую границы для интервалов [Q_1 - k * (Q_3 - Q_1), Q_3 + k * (Q_3 - Q_1)].
        Если левая граница меньше нуля, то мы заменяем ее нулем """
    q1 = volumes_.quantile(0.25)
    q3 = volumes_.quantile(0.75)
    return max(q1 - k_ * (q3 - q1), 0), q3 + k_ * (q3 - q1)


tukey_fences(transformed_datasets[0]['Объем на предыдущую дату'])

(1320350.0, 2509475.0)

In [8]:
def get_log_profit_by_volume(tdf: pd.DataFrame, low_volume: Optional[float] = None,
                             high_volume: Optional[float] = None) -> pd.Series:
    """ Возвращает данные доходности, которые попали в интервал между low и high объемом торгов """

    # tdf - transformed dataframe i
    if low_volume is None:  # Если левую границу не задали сделаем ее минимумом
        low_volume = tdf['Объем на предыдущую дату'].min()
    if high_volume is None:  # Если правую границу не задали сделаем ее максимумом
        high_volume = tdf['Объем на предыдущую дату'].max()

    tdf = tdf[(tdf['Объем на предыдущую дату'] >= low_volume) & (tdf['Объем на предыдущую дату'] < high_volume)]
    return tdf['Логарифмическая доходность']


i = 1
# Пример выделения низкого объема торгов
low, high = tukey_fences(transformed_datasets[i]['Объем на предыдущую дату'])
sample_profit_low_volume = get_log_profit_by_volume(transformed_datasets[i], 0, low)
sample_profit_low_volume.head()

3   -0.015097
4    0.018349
5    0.000000
6    0.003241
7   -0.006493
Name: Логарифмическая доходность, dtype: float64

# Формирование критериев

## Критерий Пирсона
$X_{(0)}, X_{(1)}, \dots, X_{(n)}$ - вариационный ряд выборки размера $n$ из распределения $\mathcal{L}(\theta)$. Мы будем проверять гипотезу $X \sim \mathcal{L}(\theta) = \mathcal{N}(\theta_0, \theta_1^2)$ - нормальное распределение, где $\theta_0$ - оценка математического ожидания, $\theta_1$ - оценка $\sigma$, среднеквадратического отклонения.


Сгруппируем наблюдения по $k$ непересекающимся интервалам:

$$ X_{(0)} = t_{(0)} < t_{(1)} < \dots < t_{(k)} = X_{(n)} $$

Тогда $n_i$ - количество наблюдений, попавших в полуинтервал $(t_{(i - 1)}, t_{(i)}], \quad i \geq 1$.
И обозначим вероятность $p_i = P(t_{(i - 1)} < X \leq t_{(i)}) = P(X \leq t_{(i)}) - P(X \leq t_{(i - 1)}) = F(t_{(i)}) - F(t_{(i - 1)})$


Составим статистику $\chi^2_n$ Пирсона:

$$\chi^2_n = \sum_{i = 1}^{n} \frac{(n_i - p_i \cdot n)^2}{p_i \cdot n}$$

Для нормального распределения известно, что оценки этих параметров путем использования метода максимального правдоподобия: $\theta_0 = \overline X, \theta_1^2 = \frac{1}{n} \sum_{i=1}^{n}(X_i - \overline X) ^ 2$

Эта статистика при $n \longrightarrow \infty$ подчиняется распределению $\chi^2_{k-m-1} = \chi^2_{k-2-1} $, $m$ - количество оцениваемых парамтеров.

In [9]:
ALPHA = 0.05  # Уровень значимости


def chi_stat_norm(t: np.array, x: np.array, theta0: float = 0, theta1: float = 1) -> pd.DataFrame:
    """
    Находит статистику хи-квадрат Пирсона для проверки соответствия выборки x нормальному распределению
    с параметрами theta0, theta1.

    Parameters
    ----------
    t - разбиение t_0, t_1, ..., t_k - размера k + 1
    x - выборка размера n
    theta0 - оцениваемый параметр мат. ожидания
    theta1 - оцениваемый параметр sigma

    Returns
    -------
    Возвращает статистику хи-квадрат Пирсона
    """
    eps = 1e-1  # Параметр для дополнительной группировки, с целью избежать деления на околонулевые значения

    normal_distribution = sts.norm(theta0, theta1)
    f_obs = []  # Группируем нашу выборку
    f_exp = []  # Вероятности p_i
    k_ = len(t) - 1
    n_i, p_i = 0, 0
    for j_ in range(k_ - 1):
        # Избегаем потери данных и суммы вероятностей меньше 1
        if j_ == 0:
            t[j_] = -np.inf
        elif j_ == k_ - 2:
            t[j_ + 1] = np.inf

        n_i += sum((t[j_] < x) & (x <= t[j_ + 1]))
        # norm_dist.cdf - метод, который вычисляет функцию распределения в точке F(t). Она описана выше
        p_i += normal_distribution.cdf(t[j_ + 1]) - normal_distribution.cdf(t[j_])

        if p_i > eps:
            f_obs.append(n_i)
            f_exp.append(p_i)
            n_i, p_i = 0, 0

    n_ = sum(f_obs)
    # Меняем типы списков на массивы numpy
    f_obs = np.array(f_obs)
    f_exp = np.array(f_exp) * n_

    # Вычисляем статистику хи-квадрат
    chi2 = sum((f_obs - f_exp) ** 2 / f_exp)

    # Вычисляем p-value
    p_value = sts.chi2(k_ - 2 - 1).sf(chi2)
    if p_value < ALPHA:
        message = 'Гипотеза отвергается'
    else:
        message = 'Гипотеза не отвергается'

    return pd.DataFrame({'Наблюдаемое хи-квадрат': [chi2],
                         'Процентная точка хи-квадрат': [sts.chi2(k_ - 2 - 1).isf(ALPHA)],
                         'P-value': [p_value],
                         'Вывод': [message]})


# Пример работы функции chi_stat_norm
x_sample = sample_profit_low_volume
k = 1 + int(np.log2(len(x_sample)))  # По формуле Стерджесса рассчитываем количество разбиений
t_sample = np.linspace(x_sample.quantile(0.01), x_sample.quantile(0.99), k)
chi_stat_norm(t_sample, x_sample, theta0=x_sample.mean(), theta1=x_sample.std())

,Наблюдаемое хи-квадрат,Процентная точка хи-квадрат,P-value,Вывод
0,10.399544,12.591587,0.108804,Гипотеза не отвергается


# Критерий согласия Колмогорова
Пусть $F_n(x) = \frac{1}{n}\sum_{i = 1}^{n}I_{X_k \leq x}$ - эмпирическая функция распределения. $F(x)$ - теоритическая функция распределения.

Статистика Колмогорова:
$$D_n = \sup_{x\in \mathbb{R}} \left| F_n(x) - F(x) \right|$$

Если статистика $\sqrt{n}D_n$ превышает процентную точку распределения Колмогорова ${\displaystyle K_{\alpha }}$ заданного уровня значимости $ \alpha $ , то  гипотеза $H_{0}$ (о соответствии закону $F(x)$) отвергается. Иначе гипотеза не отвергается на уровне $\alpha$.

Также статистику $D_{n, набл}$ можно вычислить с помощью формулы:
$$D_{n, набл} = \sqrt{n} \cdot \max_{k}\left\{\left|F(x_{(k)}) - \frac{2k - 1}{2n}\right| + \frac{1}{2n}\right\}$$


In [10]:
def kolmogorov_statistic_norm(x_rvs: np.ndarray, theta0: float = 0, theta1: float = 1) -> pd.DataFrame:
    """
    Вычисляет статистику Колмогорова для сравнения с нормальным распределением

    Parameters
    ----------
    x_rvs - выборка размера n
    theta0 - оцениваемый параметр мат. ожидания
    theta1 - оцениваемый параметр sigma

    Returns
    -------
    Возвращает статистику Колмогорова
    """

    from scipy.special import kolmogi, kolmogorov
    n_ = len(x_rvs)  # Размер выборки
    x_rvs = sorted(x_rvs)
    max_d = -100  # Инициализация статистики
    normal_distribution = sts.norm(theta0, theta1)
    for k_ in range(1, n_ + 1):
        max_d = max(max_d,
                    n_ ** 0.5 * (abs(normal_distribution.cdf(x_rvs[k_ - 1]) - (2 * k_ - 1) / (2 * n_)) + 1 / (2 * n_)))

    if max_d > kolmogi(ALPHA):
        message = 'Гипотеза отвергается'
    else:
        message = 'Гипотеза не отвергается'

    p_value = kolmogorov(max_d)

    return pd.DataFrame({'Статистика Колмогорова': [max_d],
                         'Процентная точка Колмогорова': [kolmogi(ALPHA)],
                         'P-value': [p_value],
                         'Вывод': [message], })


# Пример работы функции chi_stat_norm
x_sample = sample_profit_low_volume
kolmogorov_statistic_norm(x_sample, theta0=x_sample.mean(), theta1=x_sample.std())

,Статистика Колмогорова,Процентная точка Колмогорова,P-value,Вывод
0,1.266914,1.358099,0.080697,Гипотеза не отвергается


# Проверка гипотез модельных данных

Сгенерируем данные из нормального распределения 1000 раз и проверим, сколько раз гипотезы отверглись. Должно быть около 5% отвергнутых гипотез

### Критерий $\chi^2$ Пирсона

In [11]:
p_values_chi = []

for i in range(1000):
    x_norm_rvs = sts.norm().rvs(100)
    k = 1 + int(np.log2(len(x_norm_rvs)))  # По формуле Стерджесса рассчитываем количество разбиений
    t_sample = np.linspace(min(x_norm_rvs), max(x_norm_rvs), k)
    _, _, p_v, _ = chi_stat_norm(t_sample, x_norm_rvs, theta0=0, theta1=1).values[0]
    p_values_chi.append(p_v)

print(f'Процент отвергнутых гипотез: {sum(np.array(p_values_chi) <= 0.05) / 1000 :.1%}')

Процент отвергнутых гипотез: 5.5%


### Критерий Колмогорова

In [12]:
p_values_kolmogorov = []

for i in range(1000):
    x_norm_rvs = sts.norm().rvs(100)
    _, _, p_v, _ = kolmogorov_statistic_norm(x_norm_rvs, theta0=0, theta1=1).values[0]
    p_values_kolmogorov.append(p_v)

print(f'Процент отвергнутых гипотез: {sum(np.array(p_values_kolmogorov) <= 0.05) / 1000 :.1%}')

Процент отвергнутых гипотез: 4.2%


### Гистограммы распределений 



In [13]:
plots_p_values = make_subplots(1, 2, subplot_titles=['Распределение p-values. Хи-квадрат',
                                                     'Распределение p-values. Колмогоров'])
plots_p_values.add_histogram(x=p_values_chi, row=1, col=1, showlegend=False, 
                             marker={'color': 'rgb(40, 40, 200)'}, 
                             xbins={'start': 0, 'end': 1, 'size': 0.1})
plots_p_values.add_histogram(x=p_values_kolmogorov, row=1, col=2, showlegend=False,
                             marker={'color': 'rgb(40, 40, 200)'},
                             xbins={'start': 0, 'end': 1, 'size': 0.1})

plots_p_values.update_xaxes(title='P-value')
plots_p_values.update_yaxes(title='Частота')
plots_p_values.show(renderer="colab")

# Мощность критерия $\chi^2$ Пирсона
Рассмотрим альтернативную две альтернативные гипотезы
1. $H_1:$ логарифмической доходности имеет распределение стьюдента с 5 степенями свободы.
2. $H_1:$ распределение логарифмической доходности имеет распределение лапласса с параметрами 0 и 1.

Найдем мощность критерия для двух случаев. Количество торговых дней - 600 (низкий и высокие объемы торгов) и 1200 (средние объемы торгов).

In [14]:
import plotly.express as px

norm = sts.norm(0, 1).pdf
laplace = sts.laplace(0, 1).pdf
student = sts.t(5).pdf
x_axis = np.linspace(-3, 3, 500)
fig = px.line({'x': x_axis, 'Нормальное': norm(x_axis), 'Стьюдента': student(x_axis), 'Лапласа': laplace(x_axis)},
              x='x', y=['Нормальное', 'Стьюдента', 'Лапласа'], title='<b>Три распределения</b>')

fig.update_layout(legend_title_text='Распределение')
fig.update_yaxes(title={'text': 'f(x)'})
fig.show(renderer="colab")

In [15]:
# Алгоритм:
# 1. Генерируем данные из распределения стьюдента со степенью свободы 5.
# 2. Проверяем их с помощью критерия хи-квадрат
# 3. Подсчитываем количество отвержений гипотезы H0
# 4. Такой эксперимент повторяем 1000 раз для каждого набора распределения / объема

init_df = pd.DataFrame(columns=['Распределение', 'Объем', 'Мощность'])  # Инициализировали датасет
names = ['Стьюдента (5)', 'Лапласа (0, 1)']  # Названия распределений
volumes = [600, 1200]  # Объемы
distributions = [sts.t(5), sts.laplace()]  # Инициализировали распределения
m = 0  # Счетчик для новой строки

for i in range(2):
    for j in range(2):
        power = 0  # Счетчик отвержения гипотезы H0 в пользу H1
        for _ in range(1000):
            x_sample = distributions[i].rvs(volumes[j])  # Генерируем данные из распределения при условии H1
            # Вычисляем статистику
            t_sample = np.linspace(np.quantile(x_sample, 0.01), np.quantile(x_sample, 0.99), k)
            _, _, p_v, _ = chi_stat_norm(t_sample, x_sample, theta0=x_sample.mean(), theta1=x_sample.std()).values[0]
            if p_v < ALPHA:
                power += 1
        init_df.loc[m] = [names[i], volumes[j], power / 1000]
        m += 1

init_df

,Распределение,Объем,Мощность
0,Стьюдента (5),600,0.809
1,Стьюдента (5),1200,0.990
2,"Лапласа (0, 1)",600,0.999
3,"Лапласа (0, 1)",1200,1.000


Видим, что при верности гипотезы $H_1$ критерий показывает малое число ошибок второго рода при больших объемах выборки. Можем заметить, что в силу схожести распределения стьюдента и нормального распределения, особенно при большом показателе степеней свободы, мощность критерия на малом объема была самая низкая.

# Проверка реальных данных

## Построение гистограмм

In [16]:
plots = make_subplots(2, 3, subplot_titles=tickers, x_title='Логарифмическая доходность', y_title='Количество')
for i in range(6):
    data = transformed_datasets[i]
    low, high = tukey_fences(data['Объем на предыдущую дату'])
    sample_profit_volume = get_log_profit_by_volume(data, 0, low)

    plots.add_histogram(x=sample_profit_volume, row=i // 3 + 1, col=i % 3 + 1, showlegend=False,
                        marker={'color': 'rgb(40, 40, 200)'}, opacity=0.8)

plots.update_layout(title='<b>Распределение логарифмической доходности для низкого объема торгов</b>')
plots.show(renderer="colab")

In [17]:
plots = make_subplots(2, 3, subplot_titles=tickers, x_title='Логарифмическая доходность', y_title='Количество')
for i in range(6):
    data = transformed_datasets[i]
    low, high = tukey_fences(data['Объем на предыдущую дату'])
    sample_profit_volume = get_log_profit_by_volume(data, low, high)

    plots.add_histogram(x=sample_profit_volume, row=i // 3 + 1, col=i % 3 + 1, showlegend=False,
                        marker={'color': 'rgb(40, 40, 200)'}, opacity=0.8)

plots.update_layout(title='<b>Распределение логарифмической доходности для среднего объема торгов</b>')

plots.show(renderer="colab")

In [18]:
plots = make_subplots(2, 3, subplot_titles=tickers, x_title='Логарифмическая доходность', y_title='Количество')
for i in range(6):
    data = transformed_datasets[i]
    low, high = tukey_fences(data['Объем на предыдущую дату'])
    sample_profit_volume = get_log_profit_by_volume(data, high, None)

    plots.add_histogram(x=sample_profit_volume, row=i // 3 + 1, col=i % 3 + 1, showlegend=False,
                        marker={'color': 'rgb(40, 40, 200)'}, opacity=0.8)

plots.update_layout(title='<b>Распределение логарифмической доходности для высокого объема торгов</b>')
plots.show(renderer="colab")

## Применение $\chi^2$ Пирсона

In [19]:
# Построим таблицу по каждому тикеру и объема торгов. В столбцах будем указывать объем торгов, в строках индекс, на пересечении - итог применения критерия
init_df = pd.DataFrame(columns=['Тикер', 'Объем', 'P-value', 'Итог'])
m = 0  # Номер строки
for i in range(len(tickers)):  # Перебор по тикерам
    data = transformed_datasets[i]
    low, high = tukey_fences(data['Объем на предыдущую дату'])
    constraints = [(0, low), (low, high), (high, None)]
    volume_name = ['Низкий', 'Средний', 'Высокий']

    for j in range(3):  # Перебор по объемам
        sample_profit_volume = get_log_profit_by_volume(data, constraints[j][0], constraints[j][1])
        k = 1 + int(np.log2(len(sample_profit_volume)))  # По формуле Стерджесса рассчитываем количество разбиений

        t_sample = np.linspace(sample_profit_volume.quantile(0.01), sample_profit_volume.quantile(0.99), k)
        _, _, p_v, mes = chi_stat_norm(t_sample, sample_profit_volume, theta0=sample_profit_volume.mean(),
                                       theta1=sample_profit_volume.std()).values[0]
        init_df.loc[m] = [tickers[i], volume_name[j], p_v, mes]
        m += 1

chi_sq_pivot_table = init_df.pivot_table(values='Итог', index='Тикер', columns='Объем', aggfunc='first')
chi_sq_pivot_table

Объем,Высокий,Низкий,Средний
Тикер,,,
0883.HK,Гипотеза отвергается,Гипотеза отвергается,Гипотеза отвергается
0939.HK,Гипотеза отвергается,Гипотеза отвергается,Гипотеза отвергается
0941.HK,Гипотеза отвергается,Гипотеза не отвергается,Гипотеза отвергается
1398.HK,Гипотеза отвергается,Гипотеза отвергается,Гипотеза отвергается
HSBC,Гипотеза отвергается,Гипотеза отвергается,Гипотеза отвергается
TCEHY,Гипотеза не отвергается,Гипотеза отвергается,Гипотеза отвергается


In [20]:
chi_sq_p_values = init_df.pivot_table(values='P-value', index='Тикер', columns='Объем', aggfunc='first')
chi_sq_p_values = chi_sq_p_values.round(5)
chi_sq_p_values

Объем,Высокий,Низкий,Средний
Тикер,,,
0883.HK,0.00000,0.00404,0.00000
0939.HK,0.00005,0.02695,0.02852
0941.HK,0.00000,0.10880,0.00000
1398.HK,0.00031,0.00131,0.00000
HSBC,0.00000,0.00059,0.00000
TCEHY,0.65076,0.02043,0.00000


## Применение критерия Колмогорова

In [21]:
# Построим таблицу по каждому тикеру и объема торгов. В столбцах будем указывать объем торгов, в строках индекс, на пересечении - итог применения критерия
init_df = pd.DataFrame(columns=['Тикер', 'Объем', 'P-value', 'Итог'])
m = 0  # Номер строки
for i in range(len(tickers)):  # Перебор по тикерам
    data = transformed_datasets[i]
    low, high = tukey_fences(data['Объем на предыдущую дату'])
    constraints = [(0, low), (low, high), (high, None)]
    volume_name = ['Низкий', 'Средний', 'Высокий']

    for j in range(3):  # Перебор по объемам
        sample_profit_volume = get_log_profit_by_volume(data, constraints[j][0], constraints[j][1])

        _, _, p_v, mes = kolmogorov_statistic_norm(sample_profit_volume, theta0=sample_profit_volume.mean(),
                                                   theta1=sample_profit_volume.std()).values[0]
        init_df.loc[m] = [tickers[i], volume_name[j], p_v, mes]
        m += 1

kolmogorov_pivot_table = init_df.pivot_table(values='Итог', index='Тикер', columns='Объем', aggfunc='first')
kolmogorov_pivot_table

Объем,Высокий,Низкий,Средний
Тикер,,,
0883.HK,Гипотеза отвергается,Гипотеза не отвергается,Гипотеза отвергается
0939.HK,Гипотеза отвергается,Гипотеза не отвергается,Гипотеза отвергается
0941.HK,Гипотеза отвергается,Гипотеза не отвергается,Гипотеза отвергается
1398.HK,Гипотеза отвергается,Гипотеза не отвергается,Гипотеза отвергается
HSBC,Гипотеза отвергается,Гипотеза не отвергается,Гипотеза отвергается
TCEHY,Гипотеза не отвергается,Гипотеза не отвергается,Гипотеза отвергается


In [22]:
kolmogorov_p_values = init_df.pivot_table(values='P-value', index='Тикер', columns='Объем', aggfunc='first')
kolmogorov_p_values = kolmogorov_p_values.round(5)
kolmogorov_p_values

Объем,Высокий,Низкий,Средний
Тикер,,,
0883.HK,0.00035,0.08490,0.00109
0939.HK,0.01187,0.10077,0.03209
0941.HK,0.00026,0.08070,0.00205
1398.HK,0.03483,0.05020,0.00572
HSBC,0.00065,0.05384,0.00514
TCEHY,0.25392,0.17379,0.00397


# Заключение

Критерий хи-квадрат Пирсона отвергает гипотезу о нормальном распределении логарифмической доходности на китайском биржевом рынке в большинстве случаев. В более широком классе случаев критерий Колмогорова не отрицает гипотезу о нормальном распределении, включая в себя те индексы, на которых хи-квадрат тоже не дал отрицательного ответа. В целом при малых объемах торгов нормальность в логарифмической доходности прослеживается, что нельзя сказать о среднем объеме торгов накануне.